In [1]:
import sympy as sp
import numpy as np

In [181]:
# Define A, B, C matrices
A = sp.Matrix([
    [0, 4], 
    [0, 1]])
B = sp.Matrix([
    [0], 
    [10]])
C = sp.Matrix([[1, 0]])

Ts_val = 1.9            # [s]               (Tiempo de asentamiento)
os = 0.19               # [%/100]           (Overshoot)
err_perm_nulo = True
S_3 = -100              # [rad/s]           (3er polo)



# Define the symbols
s = sp.symbols('s', complex=True)
Ts, xi, wn = sp.symbols('T_S \\xi \\omega_n', positive=True, real=True)
k1, k2, ke = sp.symbols('k_1 k_2 k_e')

xi_val = sp.simplify((-sp.ln(os) / sp.sqrt(sp.pi**2 + sp.ln(os)**2))).evalf()
wn_val = sp.simplify(4 / (xi_val * Ts_val)).evalf()
S_roots = [(-xi_val*wn_val + j*wn_val*sp.sqrt(1 - xi_val**2)).evalf() for j in [sp.I, -sp.I]]
S_roots.append(S_3)

print("Response parameters:")
display(sp.Eq(xi, sp.N(xi_val, 4)), sp.Eq(wn, sp.N(wn_val, 4)))
for i, root in enumerate(S_roots):
    root = sp.N(root, 4)
    # print(f"S_{i+1}: {str(root).ljust(20)} = {np.abs(root)}  /_  {np.angle(complex(root)):0.2f} rad")
    display(sp.Eq(sp.symbols(f'S_{i+1}'), root))

desired_poly = 1
for root in S_roots:
    desired_poly *= (s - root)
desired_poly = sp.N(desired_poly, 4)

print("Desired polynomial:")
display(sp.Eq(sp.symbols('P_d'), desired_poly))
desired_poly = desired_poly.as_poly(s).as_expr()
display(sp.Eq(sp.symbols('P_d'), desired_poly, evaluate=False))

Response parameters:


Eq(\xi, 0.4673)

Eq(\omega_n, 4.505)

Eq(S_1, -2.105 + 3.983*I)

Eq(S_2, -2.105 - 3.983*I)

Eq(S_3, -100.0)

Desired polynomial:


Eq(P_d, (s + 100.0)*(s + 2.105 - 3.983*I)*(s + 2.105 + 3.983*I))

Eq(P_d, 1.0*s**3 + 104.2*s**2 + 441.3*s + 2029.0)

In [182]:
Aa = sp.MatrixSymbol('A_a', 3, 3)
Ba = sp.MatrixSymbol('B_a', 3, 1)
Ca = sp.MatrixSymbol('C_a', 3, 3)

Aa_val = sp.Matrix([[A, sp.zeros(2, 1)], [C, sp.zeros(1, 1)]])
Ba_val = sp.Matrix([B, sp.zeros(1, 1)])
Ca_val = sp.Matrix([[C, sp.zeros(1, 1)]])
k = sp.Matrix([[k1, k2, -ke]])

display(sp.Eq(Aa, Aa_val, evaluate=False), sp.Eq(Ba, Ba_val, evaluate=False), sp.Eq(Ca, Ca_val, evaluate=False))

pole_place_eq = sp.Eq( sp.Determinant(s * sp.Identity(3) - Aa + Ba*k), 0)
print("Pole placement equation:")
display(pole_place_eq)

pole_place_eq = pole_place_eq.subs(Aa, Aa_val).subs(Ba, Ba_val).subs(sp.Identity(3), sp.eye(3)).expand().doit()
print("with values")
display(pole_place_eq)
# combine the terms

pole_place_eq = sp.Eq(sp.Determinant(pole_place_eq.lhs.arg.doit()), 0)
print("combined")
display(pole_place_eq)

pole_place_eq = pole_place_eq.doit().as_poly(s).as_expr()
print("simplified")
display(sp.Eq(pole_place_eq, 0))

Eq(A_a, Matrix([
[0, 4, 0],
[0, 1, 0],
[1, 0, 0]]))

Eq(B_a, Matrix([
[ 0],
[10],
[ 0]]))

Eq(C_a, Matrix([[1, 0, 0]]))

Pole placement equation:


Eq(Determinant(s*I - A_a + B_a*Matrix([[k_1, k_2, -k_e]])), 0)

...

Eq(Determinant(Matrix([
[ 0, -4, 0],
[ 0, -1, 0],
[-1,  0, 0]]) + Matrix([
[     0,      0,       0],
[10*k_1, 10*k_2, -10*k_e],
[     0,      0,       0]]) + Matrix([
[s, 0, 0],
[0, s, 0],
[0, 0, s]])), 0)

combined


Eq(Determinant(Matrix([
[     s,             -4,       0],
[10*k_1, 10*k_2 + s - 1, -10*k_e],
[    -1,              0,       s]])), 0)

simplified


Eq(40*k_1*s - 40*k_e + s**3 + s**2*(10*k_2 - 1), 0)

In [183]:
# iterate both polynomials to get the coefficients
eqs = []
for i in range(4):
    lhs = desired_poly.as_poly(s).coeffs()[i]
    rhs = pole_place_eq.as_poly(s).coeffs()[i]
    eqs.append(sp.Eq(lhs, rhs))

sol = sp.solve(eqs, (k1, k2, ke))

print("Solution:")
for key, val in sol.items():
    display(sp.Eq(key, val))

k1_val, k2_val, ke_val = [val.evalf() for val in sol.values()]
# k_val = sp.Matrix([k1_val, k2_val, ke_val])

Solution:


Eq(k_1, 11.0325)

Eq(k_2, 10.52)

Eq(k_e, -50.725)

# Error permanente

$ e_{ss} = \lim_{s \to \infty} s R(s) \left[ 1 - C \left( s I - A_{cl} \right)^{-1} \right] $

$ e_{ss} = \left[ 1 - C \left(A_{cl} \right)^{-1} \right] $

In [180]:
# permanent error
Rs = sp.Function('R')(s)

# matrix to scalar value function
Acl = A - B * sp.Matrix([[k1_val, k2_val]])


# display(sp.Eq(inv_sym, inv, evaluate=False))

ess = sp.Limit(s*Rs*(1 - (C*(Acl).inv())[0]), s, 0)

# ess = ess.subs(Rs, 1/s)
# ess.evalf()

ess


Limit(1.62780638516993*s*R(s), s, 0, dir='+')